* https://docs.google.com/document/d/1b7DlLtvrVxihcO65dsBaIjUU-a6aLjdQfNHSOM2PhUM/edit?tab=t.0#heading=h.v0phdvfalpr2

## setup and tryout

In [8]:
%load_ext dotenv
%dotenv

```sh
pip install langchain-google-genai langchain langchain_openai beautifulsoup4 langchain-community lxml serpapi google-search-results faiss-cpu langchainhub langchain_elasticsearch wikipedia langgraph  --upgrade
```

or `-U`

In [5]:
import os

# from openai import OpenAI
import jupyter_black
import tqdm, tqdm.notebook
from IPython.display import display, Markdown, HTML

jupyter_black.load()

assert {"OPENAI_API_KEY", "GOOGLE_API_KEY"} <= set(os.environ)

GEMINI_MODEL_NAME_CLEVER = "gemini-2.0-flash"
GEMINI_MODEL_NAME_FAST = "gemini-2.0-flash-lite"

In [6]:
%%time
from langchain_google_genai import ChatGoogleGenerativeAI

# Make sure to set your GOOGLE_API_KEY environment variable
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")

response = llm.invoke("What are the best practices for developing with LangChain?")

print(response.content)

Developing with LangChain effectively involves understanding its core components, leveraging its strengths, and following best practices for maintainability, performance, and reliability. Here's a breakdown of key best practices:

**1. Understand the Core Components:**

*   **LLMs (Large Language Models):**  The foundation.  Know which LLMs are available, their strengths and weaknesses (e.g., cost, context window, speed, capabilities), and how to integrate them.
*   **Prompts:**  Crafting effective prompts is crucial.  Learn prompt engineering techniques (few-shot learning, chain-of-thought, persona development) to get the desired outputs.
*   **Chains:**  Chains are sequences of operations.  Understand the different chain types (Sequential, Router, Transform, etc.) and choose the right one for your task.
*   **Indexes:**  Indexes store and retrieve data from external sources (documents, databases).  Master the different index types (Vectorstores, Document loaders, etc.) and choose the

original:

```python
from langchain_openai.chat_models import ChatOpenAI
# chat = ChatOpenAI(openai_api_key="...")
# If you have an envionrment variable set for OPENAI_API_KEY, you can just do:
chat = ChatOpenAI()
chat.invoke("Hello, how are you?") 
```

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Make sure to set your GOOGLE_API_KEY environment variable.
# You can get one from Google AI Studio: https://aistudio.google.com/app/apikey

# Initialize the Gemini chat model
# You can also specify other models like "gemini-1.5-pro"
chat = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")
# Invoke the model with a prompt
response = chat.invoke("Hello, how are you?")
print(response.content)

I am doing well, thank you for asking! As a large language model, I don't experience emotions like humans do, but I am functioning and ready to assist you. How can I help you today?


## 85. Chat Models -- Coding

In [ ]:
from IPython.display import Markdown

response = chat.invoke("What is the capital of France?")
Markdown(response.content)

In [ ]:
response.response_metadata

original:

```python
from langchain_core.messages import HumanMessage, SystemMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]
result = chat.invoke(messages)
```

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# Ensure your GOOGLE_API_KEY environment variable is set
# 1. Initialize the Gemini chat model
chat = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")

# 2. Define the message(s) for the model
text = "What would be a good company name for a company that makes colorful socks?"
messages = [
    SystemMessage(content="You are a helpful assistant that generates company names."),
    HumanMessage(content=text),
]

# 3. Invoke the model with the messages
result = chat.invoke(messages)

# 4. Print the AI's response content
print(result.content)

## 86. Chat Prompt Templates
* https://drive.google.com/file/d/1JoyxZlYfngmXnvrRyo7qqvUoB7qz6il0/view?usp=drive_link

In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate

chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that generates company names"),
        ("human", "{text}"),
    ]
)

result = chat_prompt_template.invoke(
    {
        "text": "What would be a good company name for a company that makes colorful socks?"
    }
)

# model = Cha(model='gpt-4o-mini')

ai_llm_result = chat.invoke(result)
print(ai_llm_result.content)

## 87. Streaming
* https://drive.google.com/file/d/18sGlOZ8AKwON1CXUMnqf9ONfj7bwjSiO/view?usp=drive_link

In [ ]:
import sys
import tqdm, tqdm.notebook

chat = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    # streaming=True,
)
for chunk in tqdm.notebook.tqdm(chat.stream("What is the capital of the moon?")):
    print(chunk.content, end="", flush=True)
    sys.stdout.flush()

## 88. Output Parsers
* https://drive.google.com/file/d/1QWwi3AOCHEoMR83zR21sB7zzKdUxVdfO/view?usp=drive_link

In [ ]:
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [ ]:
class Joke(BaseModel):
    setup: str = Field(description="The setup to the joke")
    punchline: str = Field(description="The punchline to the joke")


class Jokes(BaseModel):
    jokes: List[Joke] = Field(description="A list of jokes")


parser = PydanticOutputParser(pydantic_object=Joke)

In [ ]:
print(parser.get_format_instructions())

In [ ]:
template = "Answer the user query.\n{format_instructions}\n{query}"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])


messages = chat_prompt.invoke(
    {
        "query": "What is a really funny joke about Python programming?",
        "format_instructions": parser.get_format_instructions(),
    }
)

In [ ]:
chat = ChatOpenAI()
## does not work with Gemini
result = chat.invoke(messages)

In [ ]:
try:
    joke_object = parser.parse(result.content)
    print(joke_object.setup)
    print(joke_object.punchline)
except Exception as e:
    print(e)

In [ ]:
chat = ChatOpenAI(model="gpt-4.1-mini")
structured_llm = chat.with_structured_output(Joke)
result = structured_llm.invoke("What is a really funny joke about Python programming?")

In [ ]:
result

In [ ]:
class Joke(BaseModel):
    setup: str = Field(description="The setup to the joke")
    punchline: str = Field(description="The punchline to the joke")
    explanation: str = Field(
        description="A detailed explanation of why this joke is funny."
    )


class Jokes(BaseModel):
    jokes: List[Joke] = Field(description="A list of jokes")

In [ ]:
chat = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
structured_llm = chat.with_structured_output(Joke)
result = structured_llm.invoke("What is a really funny joke about Python programming?")
result

## 89. Summarizing large amounts of text
* https://colab.research.google.com/drive/11t0e03SThhKRPq9T1M7xg6BcooBFaTkA

### crisp

In [ ]:
# from langchain_openai.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain
from langchain_core.prompts import PromptTemplate

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

llm = ChatGoogleGenerativeAI(
    temperature=0,
    model=GEMINI_MODEL_NAME_CLEVER,
)
chain = load_summarize_chain(llm, chain_type="stuff")

res = chain.invoke(docs)

In [ ]:
res["input_documents"]
Markdown(res["output_text"])

### map reduce

* problem if pages refer to each other (since summaries are done independently)

In [ ]:
from langchain.chains.mapreduce import MapReduceChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import (
    ReduceDocumentsChain,
    MapReduceDocumentsChain,
    StuffDocumentsChain,
)

In [ ]:
llm = ChatGoogleGenerativeAI(temperature=0, model=GEMINI_MODEL_NAME_CLEVER)

# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)

# map_chain:
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# Reduce
reduce_template = """The following is set of summaries:
{doc_summaries}
Take these and distill it into a final, consolidated summary of the main themes.
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [ ]:
# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

In [ ]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

In [ ]:
%%time
# print()
res = map_reduce_chain.invoke(split_docs)

In [ ]:
Markdown(res["output_text"])

### template

In [ ]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain({"input_documents": split_docs}, return_only_outputs=True)

# Page 1 --> Page 2 (Refine) --> Page 3 (Refine)

In [ ]:
for i, v in enumerate(result["intermediate_steps"]):
    display(Markdown(f"## step {i+1}\n{v}"))

In [ ]:
result["output_text"]

## 91. Document Loaders, Text Splitting, Creating LangChain Documents

https://colab.research.google.com/drive/1YdtBCggWStErmFeP5GBSmEaw04kXeKqD

In [ ]:
from bs4 import BeautifulSoup
from langchain_community.document_loaders import TextLoader
import requests

# Get this file and save it locally:
url = "https://github.com/hammer-mt/thumb/blob/master/README.md"

# Save it locally:
r = requests.get(url)

# Extract the text from the HTML:
soup = BeautifulSoup(r.text, "html.parser")
text = soup.get_text()

with open("README.md", "w") as f:
    f.write(text)

loader = TextLoader("README.md")
docs = loader.load()

In [ ]:
len(docs)

In [ ]:
from langchain_core.documents import Document

[Document(page_content="test", metadata={"test": "test"})]

In [ ]:
# Split the text into sentences:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=300,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

final_docs = text_splitter.split_documents(loader.load())
len(final_docs)

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain

In [ ]:
%%time
chain = load_summarize_chain(llm=chat, chain_type="map_reduce")
res = chain.invoke(
    {
        "input_documents": final_docs,
    }
)

In [ ]:
res.keys()
res["output_text"]

## 92. Tagging Documents
https://colab.research.google.com/drive/1Gn1IxMqz0RcOaDKVdY7cnzgik0JoQlqZ

In [ ]:
# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from langchain.document_loaders.sitemap import SitemapLoader
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
import pandas as pd

In [ ]:
sitemap_loader = SitemapLoader(web_path="https://understandingdata.com/sitemap.xml")
sitemap_loader.requests_per_second = 5
docs = sitemap_loader.load()

In [ ]:
# Schema
schema = {
    "properties": {
        "sentiment": {"type": "string"},
        "aggressiveness": {"type": "integer"},
        "primary_topic": {
            "type": "string",
            "description": "The main topic of the document.",
        },
    },
    "required": ["primary_topic", "sentiment", "aggressiveness"],
}

# LLM
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
## does not work with Gemini
# llm = ChatGoogleGenerativeAI(temperature=0, model=GEMINI_MODEL_NAME_CLEVER)
chain = create_tagging_chain(schema, llm, output_key="output")

In [ ]:
results = []

# Remove the 0:10 to run on all documents:
for index, doc in enumerate(docs[0:10]):
    print(f"Processing doc {index +1}")
    chain_result = chain.invoke({"input": doc.page_content})
    results.append(chain_result["output"])

In [ ]:
pd.DataFrame(results)

### with Pyadntic

In [ ]:
# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()

from langchain.document_loaders.sitemap import SitemapLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import create_tagging_chain_pydantic
from pydantic import BaseModel, Field
import pandas as pd


# 1. Pydantic Schema Definition
class DocumentTags(BaseModel):
    """Pydantic model for the tags to be extracted from the document."""

    sentiment: str = Field(
        description="The overall sentiment of the document (e.g., positive, negative, neutral)."
    )
    aggressiveness: int = Field(
        description="A rating from 1 to 10 of how aggressive the text is."
    )
    primary_topic: str = Field(description="The main topic of the document.")


# 2. Load Documents
# Note: This can take a moment to run
sitemap_loader = SitemapLoader(web_path="https://understandingdata.com/sitemap.xml")
sitemap_loader.requests_per_second = 5
docs = sitemap_loader.load()

In [ ]:
docs[0].metadata

In [ ]:
# 3. Initialize Gemini LLM
# Make sure your GOOGLE_API_KEY environment variable is set
# llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-2.0-flash-lite")
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# 4. Create the Pydantic Tagging Chain
# This chain is specifically designed to work with Pydantic models
chain = create_tagging_chain_pydantic(DocumentTags, llm)

results = []

# 5. Process Documents
# Using a smaller slice [0:3] for a quick demonstration
for index, doc in tqdm.notebook.tqdm(list(enumerate(docs[:10]))):
    print(f"--- Processing doc {index + 1} ---")

    # The input to invoke is the document content
    chain_result = chain.invoke({"input": doc.page_content})

    # The result is a Pydantic object, which we convert to a dict
    # Access the result via the "function" key
    # tag_data = chain_result["function"].dict()
    tag_data = chain_result["text"]
    results.append(tag_data)

    print(tag_data)

In [ ]:
# Optional: Display results in a DataFrame
df = pd.DataFrame(map(dict, results))
print("\n--- Final Results ---")
print(df)

## 93. Tracing with LangSmith
* https://colab.research.google.com/drive/1Sf-_1QP92iuJmFkykCufOYRkOB7tkliU
* https://smith.langchain.com
* https://serpapi.com/

In [ ]:
assert {"LANGCHAIN_API_KEY", "SERPAPI_API_KEY"} <= set(os.environ)

In [ ]:
import uuid

unique_id = uuid.uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"] = "LANGCHAIN_API_KEY"

# # Used by the agent in this tutorial
# os.environ["OPENAI_API_KEY"] = ""
# os.environ["SERPAPI_API_KEY"] = "SERPAPI_API_KEY"

In [ ]:
from langsmith import Client

client = Client()

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, load_tools

# llm = ChatOpenAI(temperature=0)
llm = ChatGoogleGenerativeAI(model=GEMINI_MODEL_NAME_CLEVER, temperature=0)

tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False
)

In [ ]:
import asyncio
import time
import tenacity

inputs = [
    "How many people live in canada as of 2023?",
    "who is dua lipa's boyfriend? what is his age raised to the .43 power?",
    "what is dua lipa's boyfriend age raised to the .43 power?",
    "how far is it from paris to boston in miles",
    "what was the total number of points scored in the 2023 super bowl? what is that number raised to the .23 power?",
    "what was the total number of points scored in the 2023 super bowl raised to the .23 power?",
    "how many more points were scored in the 2023 super bowl than in the 2022 super bowl?",
    "what is 153 raised to .1312 power?",
    "who is kendall jenner's boyfriend? what is his height (in inches) raised to .13 power?",
    "what is 1213 divided by 4345?",
]
results = []


async def arun(agent, input_example):
    try:
        return await agent.arun(input_example)
    except Exception as e:
        # The agent sometimes makes mistakes! These will be captured by the tracing.
        return e


@tenacity.retry(stop=tenacity.stop_after_attempt(4), wait=tenacity.wait_fixed(30))
def run(agent, input_example: str) -> str:
    return agent.invoke(input_example)


# for input_example in inputs:
#     results.append(arun(agent, input_example))
# results = await asyncio.gather(*results)

for input_example in tqdm.notebook.tqdm(inputs):
    time.sleep(2)
    results.append(run(agent, input_example))

In [ ]:
from langchain.callbacks.tracers.langchain import wait_for_all_tracers

# Logs are submitted in a background thread to avoid blocking execution.
# For the sake of this tutorial, we want to make sure
# they've been submitted before moving on. This is also
# useful for serverless deployments.
wait_for_all_tracers()

In [ ]:
## gemini
import pandas as pd
from IPython.display import HTML

HTML(pd.DataFrame(results).to_html())

tracing result: https://drive.google.com/drive/folders/1_4hnRpTYZxO_JLBDDE2HCESQW0eHE6Bj

## 94. LangChain Hub
* https://colab.research.google.com/drive/1lxCk4cnk60rzmu0Wz6pzK-sUmWGca5b0
* https://docs.smith.langchain.com/prompt_engineering/how_to_guides#prompt-hub

In [ ]:
from langchain import hub

In [ ]:
prompt = hub.pull("homanp/question-answer-pair")
prompt_two = hub.pull("gitmaxd/synthetic-training-data")
prompt_three = hub.pull("rlm/text-to-sql")
rag_prompt = hub.pull("rlm/rag-prompt")

In [ ]:
prompt

In [ ]:
prompt_two

In [ ]:
rag_prompt

In [ ]:
print(rag_prompt.messages)

In [ ]:
# Load docs
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

# Store splits
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

# RAG prompt
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

# LLM
from langchain.chains import RetrievalQA
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vectorstore.as_retriever(), chain_type_kwargs={"prompt": prompt}
)
question = "What are the approaches to Task Decomposition?"
result = qa_chain.invoke({"query": question})
result["result"]

## 95. LCEL (=LangChain Expression Language) - The Runnable Protocol
* https://colab.research.google.com/drive/1iHmhKEhntUy71C_gO4kNqylZQYshAgD1
* https://python.langchain.com/docs/concepts/lcel/

In [ ]:
from langchain_core.runnables import RunnableLambda

print(
    type(RunnableLambda(lambda x: x + 1))
)  # <class 'langchain.schema.runnable.RunnableLambda'>

In [ ]:
chain = RunnableLambda(lambda x: x + 1)

In [ ]:
chain.invoke(1), chain.invoke(42)

In [ ]:
# A RunnableSequence constructed using the `|` operator
sequence = RunnableLambda(lambda x: x + 1) | (lambda x: x * 2)

print(type(sequence))  # <class 'langchain.schema.runnable.RunnableSequence'>
print("\n\n---")
print(sequence.invoke(1))  # 4
sequence.batch([1, 2, 3])  # [4, 6, 8]

In [ ]:
# A sequence that contains a RunnableParallel constructed using a dict literal
sequence = RunnableLambda(lambda x: x + 1) | {
    "mul_2": RunnableLambda(lambda x: x * 2),
    "mul_5": RunnableLambda(lambda x: x * 5),
}
sequence.invoke(1)  # {'mul_2': 4, 'mul_5': 10}

In [ ]:
sequence = (
    RunnableLambda(lambda x: x + 1)
    | {
        "mul_2": RunnableLambda(lambda x: x * 2),
        "mul_5": RunnableLambda(lambda x: x * 5),
    }
    | RunnableLambda(lambda x: x["mul_2"] + x["mul_5"])
)
sequence.invoke(1)  # {'mul_2': 4, 'mul_5': 10}

In [ ]:
from langchain_core.runnables import RunnableParallel

parallel = RunnableParallel(
    {"mul_2": RunnableLambda(lambda x: x * 2), "mul_5": RunnableLambda(lambda x: x * 5)}
)

# This is a dictionary, however it will be composed with other runnables when used in a sequence:
parallel_two = {
    "mul_2": RunnableLambda(lambda x: x["input_one"] * 2),
    "mul_5": RunnableLambda(lambda x: x["input_two"] * 5),
}

print(type(parallel))  # <class 'langchain.schema.runnable.RunnableParallel'>
print(type(parallel_two))  # <class 'dict'>

In [ ]:
chain = parallel | RunnableLambda(lambda x: x["mul_2"] + x["mul_5"])
chain.invoke(5)

In [ ]:
second_chain = parallel_two | RunnableLambda(lambda x: x["mul_2"] + x["mul_5"])
second_chain.invoke({"input_one": 5, "input_two": 10})

## 96. ChatModels, itemgetter and RAG
* https://colab.research.google.com/drive/1PwupkZARnPadd4i1700WY6Y0u8eiiHxz

In [ ]:
from langchain_core.runnables import (
    RunnablePassthrough,
    RunnableParallel,
    RunnableLambda,
)
import operator

In [ ]:
runnable = RunnableParallel(origin=RunnablePassthrough(), modified=lambda x: x + 1)

print(runnable.invoke(1))  # {'origin': 1, 'modified': 2}


def fake_llm(prompt: str) -> str:  # Fake LLM for the example
    return prompt + " world"


chain = RunnableLambda(fake_llm) | {
    "original": RunnablePassthrough(),  # Original LLM output
    "parsed": lambda text: text[::-1],  # Parsing logic
}

chain.invoke("hello")

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# chat = ChatOpenAI()
chat = ChatGoogleGenerativeAI(model=GEMINI_MODEL_NAME_FAST)
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")

chain = prompt | chat
print(chain)

In [ ]:
print("first", chain.first)
print("last", chain.last)

In [ ]:
# Stream:
print("\n\nStream:\n")
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True)

# Invoke:
print("\n\nInvoke:\n")
print(chain.invoke({"topic": "bears"}).content)

# Batch:
print("\n\nBatch:\n")
res = chain.batch([{"topic": "bears"}, {"topic": "redhats"}, {"topic": "monks"}])
print(res)
print("\n".join((map(operator.attrgetter("content"), res))))

### RAG in LCEL

In [ ]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores.faiss import FAISS

In [ ]:
vectorstore = FAISS.from_texts(
    [
        "James Phoenix works as a data engineering and LLM consultant at JustUnderstandingData",
        "James Phoenix has an age of 31 years old.",
    ],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# model = ChatOpenAI()
model = ChatGoogleGenerativeAI(model=GEMINI_MODEL_NAME_CLEVER)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# It's the same as this, but the tuple allows for line breaks:
# {"context": retriever, "question": RunnablePassthrough()} | prompt | model | StrOutputParser()

In [ ]:
chain.invoke("What company does James phoenix work at?")

In [ ]:
chain.invoke("What is James Phoenix's age?")

### itemgetter

In [ ]:
test = {"data": ["This is a test", "Another entry..."]}

print(itemgetter(test))
print(itemgetter("data")(test))

In [ ]:
prompt = ChatPromptTemplate.from_template(
    """What is the profession of James Phoenix? His profession is {profession}."""
)

first_chain = RunnableParallel(name=lambda x: "James Phoenix", age=lambda x: 31)

second_chain = {
    # itemgetter is used to get the value from the dictionary from the previous step: (note this is only the previous step, not the whole chain)
    "name": itemgetter("name"),
    "age": itemgetter("age"),
    # You can not use string values, either use itemgetter or a lambda, or RunnablePassthrough
    "profession": lambda x: "Data Engineer",
}

chain = (
    first_chain
    | second_chain
    | prompt
    | ChatGoogleGenerativeAI(model=GEMINI_MODEL_NAME_FAST)
    | StrOutputParser()
)
chain.invoke({})

## 97. LCEL - Chat Message History and Memory
* https://colab.research.google.com/drive/1dmDHw39-5NNiQtz3s_b8470lXTJYNx8I

In [ ]:
from langchain_core.runnables import RunnableMap, RunnablePassthrough, RunnableLambda
from langchain_core.prompts import format_document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
from operator import itemgetter
from langchain_community.vectorstores.faiss import FAISS
from langchain_openai import OpenAIEmbeddings

### conversational history

In [ ]:
vectorstore = FAISS.from_texts(
    [
        "James Phoenix works as a data engineering and LLM consultant at JustUnderstandingData",
        "James is 31 years old.",
    ],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

In [ ]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)
CONDENSE_QUESTION_PROMPT

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)
ANSWER_PROMPT

In [ ]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs: list,
    document_prompt: PromptTemplate = DEFAULT_DOCUMENT_PROMPT,
    document_separator: str = "\n\n",
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [ ]:
from typing import List, Union
from langchain.schema import HumanMessage, SystemMessage, AIMessage


def _format_chat_history(
    chat_history: List[Union[HumanMessage, SystemMessage, AIMessage]]
) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, HumanMessage):
            buffer += "\nHuman: " + dialogue_turn.content
        elif isinstance(dialogue_turn, AIMessage):
            buffer += "\nAssistant: " + dialogue_turn.content
        elif isinstance(dialogue_turn, SystemMessage):
            buffer += "\nSystem: " + dialogue_turn.content
    return buffer

In [ ]:
chat = ChatGoogleGenerativeAI(model=GEMINI_MODEL_NAME_CLEVER, temperature=0)
_inputs = RunnableMap(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: _format_chat_history(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    # | ChatOpenAI(temperature=0)
    | chat
    | StrOutputParser(),
)
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | chat | StrOutputParser()

In [ ]:
conversational_qa_chain.invoke(
    {
        "question": "where did James work?",
        "chat_history": [],
    }
)

### memory

In [ ]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [ ]:
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    # | ChatOpenAI(temperature=0)
    | chat
    | StrOutputParser(),
}
# Now we retrieve the documents

# This is REALLY IMPORTANT as the chain above becomes StrOutputParser() so it will only have one key, which gets passed to the retriever!
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": (
        final_inputs
        | ANSWER_PROMPT
        # | ChatOpenAI()
        | chat
    ),
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = (
    loaded_memory
    | standalone_question
    | retrieved_documents
    | answer
    # | StrOutputParser()
)

In [ ]:
inputs = {"question": "where did James Phoenix work?"}
result = final_chain.invoke(inputs)
print(result)

In [ ]:
memory.save_context(inputs, {"answer": result["answer"].content})

In [ ]:
memory.load_memory_variables({})

## 98. LCEL - Multiple Chains
* https://colab.research.google.com/drive/1W6T-iQ835IEV4fPPiYykY29yCYL8ZIwa

In [ ]:
prompt1 = ChatPromptTemplate.from_template("What city was {person} born in?")
prompt2 = ChatPromptTemplate.from_template(
    "What country is the city {city} in? Respond in {language}"
)

# model = ChatOpenAI()
model = ChatGoogleGenerativeAI(model="gemini-2.5-pro")

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

In [ ]:
(
    prompt1 | ChatGoogleGenerativeAI(model=GEMINI_MODEL_NAME_FAST) | StrOutputParser()
).invoke({"person": "Barack Obama"})

In [ ]:
(prompt1 | model | StrOutputParser()).invoke({"person": "Barack Obama"})

In [ ]:
chain2.invoke({"person": "Barack Obama", "language": "Spanish"})

## 99. LCEL - Conditional Logic, Branching and Merging
* https://colab.research.google.com/drive/1f4rSRMDzzAGlSh2zR0oJGIqW2s8NfJeb

In [ ]:
from operator import itemgetter
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch, RunnablePassthrough

In [ ]:
branch = RunnableBranch(
    (lambda x: x == "hello", lambda x: x),
    (lambda x: isinstance(x, str), lambda x: x.upper()),
    (lambda x: "This is the default case, in case no above lambda functions match."),
)

print(branch.invoke("hello"))  # "hello"
print(branch.invoke("hell"))
print(branch.invoke(None))  # "This is the default case"

In [ ]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [ ]:
chain.invoke({"input": "scrum"})

In [ ]:
import logging
import langchain
from contextlib import contextmanager
from operator import itemgetter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks import BaseCallbackHandler
from typing import Any, Dict

# 1. Configure logging to save output to a file
# logging.basicConfig(
#     filename="langchain_debug.log",
#     filemode="w",
#     level=logging.INFO,
#     format="%(asctime)s - %(levelname)s - %(message)s",
# )
logger = logging.getLogger("langchain")
logger.setLevel(logging.INFO)  # Set the desired logging level (e.g., INFO, DEBUG)

# 2. Create a handler to write to a file
#    'w' mode overwrites the file each time, use 'a' to append
handler = logging.FileHandler("langchain.log", mode="w")

# 3. Create a formatter to define the log message's structure
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
handler.setFormatter(formatter)

# 4. Add the handler to the logger
logger.addHandler(handler)

# 5. Prevent logs from propagating to the root logger to avoid duplicates
logger.propagate = False


class MyLoggingCallbackHandler(BaseCallbackHandler):
    """A callback handler that logs events to a given logger."""

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any
    ) -> None:
        """Log the start of a chain run."""
        self.logger.info(f"Chain started with inputs: {inputs}")

    def on_llm_end(self, response, **kwargs: Any) -> None:
        """Log the end of an LLM call."""
        # The actual response object structure may vary by model provider
        self.logger.info(
            f"LLM generated response: {response.generations[0][0].text[:80]}..."
        )


llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")
chain = (
    ChatPromptTemplate.from_template("Tell me a joke about {topic}")
    | llm
    | StrOutputParser()
)

# Instantiate your handler with your logger
my_handler = MyLoggingCallbackHandler(logger=logger)

print("--- Running chain with a custom callback handler ---")
response = chain.invoke(
    {"topic": "cats"}, config={"callbacks": [my_handler]}  # Pass the handler here
)
print("\n--- Final Response ---")
print(response)

In [ ]:
langchain.debug

In [ ]:
import langchain

langchain.debug = False

# from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks.stdout import StdOutCallbackHandler

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")

joke_chain = (
    ChatPromptTemplate.from_template("Tell me a joke about {topic}")
    | llm
    | StrOutputParser()
    | {"joke": RunnablePassthrough(), "topic": RunnablePassthrough()}
)

explain_joke = (
    ChatPromptTemplate.from_template("Explain the joke: {joke}")
    | llm
    | StrOutputParser()
)

benefits_of_joke = (
    ChatPromptTemplate.from_template("List the benefits of this joke: {joke}")
    | llm
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are responsible for generating a small analysis of a joke. The topic will be: {topic}",
            ),
            ("ai", "{joke}. The benefits of this joke are: {benefits}"),
            ("human", "The explanation of the joke is: {explanation}"),
            ("human", "Generate a small analysis of the joke. Analysis: "),
        ]
    )
    | llm
    | StrOutputParser()
)

final_chain = (
    {"topic": RunnablePassthrough()}
    | joke_chain
    | {
        "explanation": explain_joke,
        "benefits": benefits_of_joke,
        "joke": itemgetter("joke"),
        "topic": itemgetter("topic"),
    }
    | final_responder
)

final_chain.invoke(
    {"topic": "bears"},
    # config={"callbacks": [StdOutCallbackHandler()]},
)

## 100. LCEL -- LangChain Vector Databases + indexing API
* https://colab.research.google.com/drive/1a3WMSxKRkyyGzlWf13zGrtXW6QdqQNhC

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS

In [ ]:
raw_text = """
Digital marketing encompasses a broad range of marketing activities that utilize digital channels to connect with customers. At its core, digital marketing aims to reach a targeted audience through various online and electronic means, including social media, email, search engines, and websites. Unlike traditional marketing methods, digital marketing offers unparalleled opportunities for businesses to engage with their audience in real-time, enabling personalized communication and immediate feedback. This real-time interaction not only enhances customer experience but also allows businesses to gather valuable data on consumer behaviors, preferences, and trends, facilitating more effective marketing strategies and campaigns.
The rise of digital marketing can be attributed to the increasing reliance on the internet and digital devices by consumers. As more people spend time online, businesses have shifted their marketing efforts to where their audiences are. Digital marketing leverages this online presence, employing strategies such as search engine optimization (SEO), content marketing, pay-per-click (PPC) advertising, and social media marketing to improve visibility and attract potential customers. These strategies are designed to increase traffic to a company's online platforms, build brand awareness, and ultimately drive conversions and sales. The ability to measure the effectiveness of these strategies through analytics and metrics further underscores the advantage of digital marketing, enabling businesses to refine their approach and maximize return on investment (ROI).
"""

with open("test.txt", "w") as f:
    f.write(raw_text)

In [ ]:
# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader("test.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [ ]:
db.similarity_search_with_relevance_scores("digital marketing", k=1)

In [ ]:
# Adding on extra documents directly within LangChain:
from langchain_core.documents import Document

docs = [
    Document(
        page_content="James phoenix worked in digital marketing for 3 years.",
        metadata={"source": "James Phoenix"},
    ),
    Document(
        page_content="Digital marketing is a growing industry.",
        metadata={"source": "Wikipedia"},
    ),
]

In [ ]:
db.add_documents(docs)

In [ ]:
db.similarity_search("James", k=1)

## 101. LCEL - Configurable Fields
* https://colab.research.google.com/drive/1OQ-GNTMVsPhQwQs7NbxZ5RSETNst_6Pk

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(temperature=0).configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM Temperature",
        description="The temperature of the LLM",
    )
)

In [ ]:
model.invoke("pick a random number").content

In [ ]:
model.with_config(configurable={"llm_temperature": 0.9}).invoke(
    "pick a random number"
).content

### configuring prompts

In [ ]:
# llm = ChatOpenAI(temperature=0)
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")
prompt = PromptTemplate.from_template(
    "Tell me a joke about {topic}"
).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="prompt"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="joke",
    # This adds a new option, with name `poem`
    poem=PromptTemplate.from_template("Write a short poem about {topic}"),
    # You can add more configuration options here
)
chain = prompt | llm

In [ ]:
# By default it will write a joke
chain.invoke({"topic": "bears"}).content

In [ ]:
%%time
Markdown(
    chain.with_config(configurable={"prompt": "poem"})
    .invoke({"topic": "bears"})
    .content
)

### saving configurations

In [ ]:
openai_joke = chain.with_config(configurable={"llm": "openai"})

In [ ]:
openai_joke

## 102. LCEL -- LangChain Agents & Tools
* https://colab.research.google.com/drive/1PkT9FIEtrvbIDcS_QQE8jtYTNCaPCBzQ
* https://python.langchain.com/docs/how_to/#agents

In [ ]:
# 1. Standard Tools
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

print(tool.name)
print(tool.description)
print(tool.args)

# We can see if the tool should return directly to the user
print(
    "Will this automatically return the output to the user? This value is a boolean:",
    tool.return_direct,
)

In [ ]:
tool.invoke("What is Digital Marketing?")

### tools

In [ ]:
# Import things that are needed generically
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [ ]:
@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"


print(search.name)
print(search.description)
print(search.args)

In [ ]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")


@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

In [ ]:
def search_function(query: str):
    return "LangChain"


search = StructuredTool.from_function(
    func=search_function,
    name="Search",
    description="useful for when you need to answer questions about current events",
    # coroutine= ... <- you can specify an async method if desired as well
)

### agents

In [ ]:
from langchain.agents import tool


# 1. Create the tool:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


# 2. Assign the tools to a Python list:
tools = [get_word_length]

In [ ]:
# 3. Create the ChatPromptTemplate:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [ ]:
from langchain_openai import ChatOpenAI

# 4. Create the LLM and bind the tools directly to the LLM:
llm = ChatOpenAI(model="gpt-4-turbo")
llm_with_tools = llm.bind_tools(tools=tools)

In [ ]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

# 5. Creating the LCEL agent chain:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
list(agent_executor.stream({"input": "How many letters in the word data"}))

In [ ]:
agent_executor.invoke({"input": "How many letters in the word data"})

### adding memory

In [ ]:
from langchain_core.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [ ]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
input1 = "how many letters in the word data?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})

In [ ]:
len(chat_history)

#### customizing memory with id

In [ ]:
# Customising the memory by
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store:dict = {}

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = (
    {
        "input": lambda x: x["input"],
        "history": lambda x: x.get("history", []),
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [ ]:
with_message_history.invoke(
    {"input": "My name is James", "history": []},
    config={"configurable": {"session_id": "some_session_id"}},
)

In [ ]:
with_message_history.invoke(
    {"input": "What is my name?"},
    config={"configurable": {"session_id": "some_session_id"}},
)

In [ ]:
with_message_history.invoke(
    {"input": "What is my name?"},
    config={"configurable": {"session_id": "some_different_session_id"}},
)

## 103. 2025-08-30 LangGraph Intro

## 104. Simple LangGraph Flows
* https://colab.research.google.com/drive/1bJ7p5TSFCv2lWAv-ZtFPOrRoOpj3Lw2h

In [9]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")


def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)